In [ ]:
! docker run --name postgres0 -d  -p 5438:5432 -e POSTGRES_HOST_AUTH_METHOD=trust postgres

In [1]:
! docker logs postgres0 --tail 6

2022-11-25 19:14:56.581 UTC [156] LOG:  checkpoint starting: time
2022-11-25 19:14:59.240 UTC [156] LOG:  checkpoint complete: wrote 27 buffers (0.2%); 0 WAL file(s) added, 0 removed, 0 recycled; write=2.626 s, sync=0.025 s, total=2.659 s; sync files=20, longest=0.022 s, average=0.002 s; distance=115 kB, estimate=270 kB
2022-11-25 19:29:56.451 UTC [156] LOG:  checkpoint starting: time
2022-11-25 19:29:58.682 UTC [156] LOG:  checkpoint complete: wrote 23 buffers (0.1%); 0 WAL file(s) added, 0 removed, 0 recycled; write=2.217 s, sync=0.006 s, total=2.231 s; sync files=19, longest=0.003 s, average=0.001 s; distance=114 kB, estimate=254 kB
2022-11-25 19:44:56.756 UTC [156] LOG:  checkpoint starting: time
2022-11-25 19:44:59.076 UTC [156] LOG:  checkpoint complete: wrote 24 buffers (0.1%); 0 WAL file(s) added, 0 removed, 0 recycled; write=2.311 s, sync=0.005 s, total=2.321 s; sync files=19, longest=0.002 s, average=0.001 s; distance=113 kB, estimate=240 kB


In [2]:
from pymilvus import connections
import psycopg2
connections.connect(host='localhost', port='19530')
conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
cursor = conn.cursor()

In [3]:
TABLE_NAME = "test5"
field_name = "example_test5"
from pymilvus import Collection, CollectionSchema, FieldSchema, DataType
pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
field = FieldSchema(name=field_name, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="example collection")
collection = Collection(name=TABLE_NAME, schema=schema)

In [4]:
index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }
collection.create_index(field_name=field_name, index_params=index_param)

Status(code=0, message=)

In [5]:
#Deleting previouslny stored table for clean run
drop_table = "DROP TABLE IF EXISTS " + TABLE_NAME
cursor.execute(drop_table)
conn.commit()

try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (ids bigint, title text, text text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

create postgres table successfully!


In [6]:
from IPython.display import Image, HTML, display
import pyspark
# check PySpark is running
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
conf = pyspark.SparkConf()
conf.set('spark.driver.memory','4g')
sc =  pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/25 15:13:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [7]:
spark

In [8]:
'''from IPython.display import Image, HTML, display
# check PySpark is running
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate() 
spark = SparkSession(sc)
'''

'''
conf = pyspark.SparkConf()
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
'''


"\nconf = pyspark.SparkConf()\nconf.set('spark.driver.memory','4g')\nsc = pyspark.SparkContext(conf=conf)\n"

In [9]:
PATH_TO_DATA = "../milvus_test/data"

In [10]:
covid_df = spark.read.csv(PATH_TO_DATA + "/metadata.csv", header=True, inferSchema=True)

In [11]:
print("Number of articles: {}".format(covid_df.count()))
print("Sample of articles:")
covid_df.show(5)

Number of articles: 1056660
Sample of articles:
+--------+--------------------+--------+--------------------+--------------------+--------+---------+-------+--------------------+------------+--------------------+--------------+------+----------------+--------+--------------------+--------------------+--------------------+-----+
|cord_uid|                 sha|source_x|               title|                 doi|   pmcid|pubmed_id|license|            abstract|publish_time|             authors|       journal|mag_id|who_covidence_id|arxiv_id|      pdf_json_files|      pmc_json_files|                 url|s2_id|
+--------+--------------------+--------+--------------------+--------------------+--------+---------+-------+--------------------+------------+--------------------+--------------+------+----------------+--------+--------------------+--------------------+--------------------+-----+
|ug7v899j|d1aafb70c066a2068...|     PMC|Clinical features...|10.1186/1471-2334...|PMC35282| 11472636|  no-

In [12]:
covid_df.filter(covid_df.title.isNull()).count()

503

In [13]:
# rows with no abstract - 235202 rows
covid_df= covid_df.filter(covid_df.title.isNotNull())

In [14]:
covid_df.count()

1056157

In [15]:
# we only require title, abstract
cols_to_keep = ['title', 'doi', 'abstract']
covid_df = covid_df[[cols_to_keep]]

In [16]:
covid_df.show()

+--------------------+--------------------+--------------------+
|               title|                 doi|            abstract|
+--------------------+--------------------+--------------------+
|Clinical features...|10.1186/1471-2334...|OBJECTIVE: This r...|
|Nitric oxide: a p...|        10.1186/rr14|Inflammatory dise...|
|Surfactant protei...|        10.1186/rr19|Surfactant protei...|
|Role of endotheli...|        10.1186/rr44|Endothelin-1 (ET-...|
|Gene expression i...|        10.1186/rr61|Respiratory syncy...|
|Sequence requirem...|10.1093/emboj/20....|Nidovirus subgeno...|
|Debate: Transfusi...|       10.1186/cc987|Recent evidence s...|
|The 21st Internat...|      10.1186/cc1013|The 21st Internat...|
|Heme oxygenase-1 ...|10.1186/1465-9921...|Heme oxygenase-1 ...|
|Technical Descrip...| 10.1197/jamia.m1345|This report descr...|
|Conservation of p...|10.1093/emboj/19....|Regulation of orn...|
|Heterogeneous nuc...|10.1093/emboj/19....|Heterogeneous nuc...|
|A Method to Ident...|   

In [17]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
model

SentenceTransformer(
  (0): Transformer(
    (auto_model): MPNetModel(
      (embeddings): MPNetEmbeddings(
        (word_embeddings): Embedding(30527, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): MPNetEncoder(
        (layer): ModuleList(
          (0): MPNetLayer(
            (attention): MPNetAttention(
              (attn): MPNetSelfAttention(
                (q): Linear(in_features=768, out_features=768, bias=True)
                (k): Linear(in_features=768, out_features=768, bias=True)
                (v): Linear(in_features=768, out_features=768, bias=True)
                (o): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          

In [19]:
covid_title_data=list(covid_df.select('title').toPandas()['title'])

In [20]:
covid_title_data

['Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia',
 'Nitric oxide: a pro-inflammatory mediator in lung disease?',
 'Surfactant protein-D and pulmonary host defense',
 'Role of endothelin-1 in lung disease',
 'Gene expression in epithelial cells in response to pneumovirus infection',
 'Sequence requirements for RNA strand transfer during nidovirus discontinuous subgenomic RNA synthesis',
 'Debate: Transfusing to normal haemoglobin levels will not improve outcome',
 'The 21st International Symposium on Intensive Care and Emergency Medicine, Brussels, Belgium, 20-23 March 2001',
 'Heme oxygenase-1 and carbon monoxide in pulmonary medicine',
 'Technical Description of RODS: A Real-time Public Health Surveillance System',
 'Conservation of polyamine regulation by translational frameshifting from yeast to mammals',
 'Heterogeneous nuclear ribonucleoprotein A1 regulates RNA synthesis of a cytoplasmic virus',
 "A

In [21]:
covid_doi_data=list(covid_df.select('doi').toPandas()['doi'])

In [22]:
covid_doi_data

['10.1186/1471-2334-1-6',
 '10.1186/rr14',
 '10.1186/rr19',
 '10.1186/rr44',
 '10.1186/rr61',
 '10.1093/emboj/20.24.7220',
 '10.1186/cc987',
 '10.1186/cc1013',
 '10.1186/1465-9921-4-7',
 '10.1197/jamia.m1345',
 '10.1093/emboj/19.8.1907',
 '10.1093/emboj/19.17.4701',
 '10.1251/bpo66',
 '10.1093/emboj/19.15.3932',
 '10.1186/1479-5876-2-3',
 '10.1186/1742-4690-1-11',
 '10.1186/cc2850',
 '10.1186/1471-2458-4-34',
 '10.1186/1465-9921-5-16',
 '10.1186/1471-2164-6-1',
 '10.1186/1743-8462-1-7',
 '10.1186/1475-2859-4-2',
 '10.1093/nar/gni004',
 '10.1093/nar/gni034',
 '10.1155/s1110724304307084',
 '10.1186/1471-230x-5-8',
 '10.1371/journal.pbio.0030089',
 '10.1093/nar/gki329',
 '10.1093/nar/gni053',
 '10.1093/nar/gni054',
 '10.1093/nar/gki521',
 '10.1186/1743-8462-2-7',
 '10.1186/1476-072x-4-10',
 '10.1186/1744-8603-1-1',
 '10.1093/emboj/16.5.1122',
 '10.1093/nar/gni106',
 '10.1186/1743-422x-2-48',
 '10.1093/nar/gki789',
 '10.1093/nar/gki896',
 '10.1186/1745-7580-1-2',
 '10.1186/1471-2199-6-21',

In [21]:
covid_abstract_data=list(covid_df.select('abstract').toPandas()['abstract'])

22/11/25 14:18:24 ERROR Executor: Exception in task 7.0 in stage 15.0 (TID 92)
java.lang.OutOfMemoryError: Java heap space
22/11/25 14:18:24 ERROR Executor: Exception in task 6.0 in stage 15.0 (TID 91)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3537)
	at java.base/java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:100)
	at java.base/java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:130)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1891)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1800)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1197)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:358)
	at org.apache.spark.serializer.JavaSerializationStream.writeO

Exception in thread "task-result-getter-3" Exception in thread "task-result-getter-2" Exception in thread "task-result-getter-1" java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1048)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:242)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:258)
	at scala.concurrent.impl.Promise$DefaultPromise.result(Promise.scala:263)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:293)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:103)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1154)
	at org.apache.spark.storage.BlockManager.$anonfun$getRemoteBlock$8(BlockManager.scala:1098)
	at scala.Option.orElse(Option.scala:447)
	at org.apache.spark.storage.BlockManager.getRemoteBlock(BlockManager.scala:1098)
	

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
covid_abstract_data

In [ ]:
'''from sklearn.preprocessing import normalize
sentence_em = model.encode(covid_title_data)
sentence_em = normalize(sentence_em)'''

In [23]:
import pickle
pickle_out = open("model.pkl", "wb")
pickle.dump(model, pickle_out)
pickle_out.close()

In [24]:
model_rdd_pkl = sc.binaryFiles("model.pkl")
model_rdd_data = model_rdd_pkl.collect()

# Load and broadcast python object over spark nodes
model_ = pickle.loads(model_rdd_data[0][1])

In [25]:
bc_model = sc.broadcast(model_)

In [26]:
print (bc_model.value)

SentenceTransformer(
  (0): Transformer(
    (auto_model): MPNetModel(
      (embeddings): MPNetEmbeddings(
        (word_embeddings): Embedding(30527, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): MPNetEncoder(
        (layer): ModuleList(
          (0): MPNetLayer(
            (attention): MPNetAttention(
              (attn): MPNetSelfAttention(
                (q): Linear(in_features=768, out_features=768, bias=True)
                (k): Linear(in_features=768, out_features=768, bias=True)
                (v): Linear(in_features=768, out_features=768, bias=True)
                (o): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          

In [27]:
def get_embeddings(title):
    sentence_embeddings = bc_model.value.encode(title)
    return sentence_embeddings.tolist()

In [28]:
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, IntegerType,FloatType
from pyspark.ml.linalg import Vectors, VectorUDT
emb_udf = f.udf(get_embeddings, ArrayType(FloatType()))

In [23]:
#emb_udf(f.col("title"))

Column<'get_embeddings(title)'>

In [30]:
covid_df_embeddings = covid_df.withColumn("title_embedding", emb_udf(f.col("title")))

In [31]:
covid_df_embeddings.show()

22/11/25 15:17:05 WARN PythonUDFRunner: Detected deadlock while completing task 0.0 in stage 16 (TID 85): Attempting to kill Python Worker
+--------------------+--------------------+--------------------+--------------------+
|               title|                 doi|            abstract|     title_embedding|
+--------------------+--------------------+--------------------+--------------------+
|Clinical features...|10.1186/1471-2334...|OBJECTIVE: This r...|[0.022760285, -0....|
|Nitric oxide: a p...|        10.1186/rr14|Inflammatory dise...|[-0.0751733, 0.13...|
|Surfactant protei...|        10.1186/rr19|Surfactant protei...|[-0.08223474, 0.0...|
|Role of endotheli...|        10.1186/rr44|Endothelin-1 (ET-...|[-0.12924056, -0....|
|Gene expression i...|        10.1186/rr61|Respiratory syncy...|[-0.019296607, 0....|
|Sequence requirem...|10.1093/emboj/20....|Nidovirus subgeno...|[0.104730494, 0.0...|
|Debate: Transfusi...|       10.1186/cc987|Recent evidence s...|[-0.034642506, -0...|
|

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 1 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [ ]:
covid_title_embeddings=list(covid_df_embeddings.select('title_embedding').toPandas()['title_embedding'])


In [29]:
#covid_title_embeddings = [row.title_embeddings for row in covid_df_embeddings.collect()]

In [32]:
#covid_title_embeddings = covid_df_embeddings.select("title_embedding").rdd.flatMap(lambda x: x).collect()

In [ ]:
covid_title_embeddings

In [ ]:
mr = collection.insert(covid_title_embeddings)
ids = mr.primary_keys
dicts ={}

In [ ]:
import numpy as np

In [ ]:
covid_title_array = np.array(covid_df_embeddings.select("title_embedding").collect()) 

In [ ]:
covid_title_array

In [ ]:
print(type(covid_title_array))

In [ ]:
em =list(covid_title_array)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

## Random tests 

In [ ]:
pandasDF = covid_df_embeddings.toPandas()

In [ ]:
pandasDF

In [ ]:
em1

In [ ]:
mr = collection.insert([em])

In [ ]:
em =list(sentence_em)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

In [ ]:
#covid_abstract_data=list(covid_df.select('abstract').toPandas()['abstract'])

In [ ]:
# Get questions and answers.
data = pd.read_csv('data/example.csv')
title_data = data['title'].tolist()
text_data = data['text'].tolist()

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

model = SentenceTransformer('paraphrase-mpnet-base-v2')
# Get questions and answers.
data = pd.read_csv('data/example.csv')
title_data = data['title'].tolist()
text_data = data['text'].tolist()

sentence_embeddings = model.encode(title_data)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

In [ ]:
title_data

In [ ]:
text_data

In [ ]:
sentence_embeddings

In [ ]:
em =list(sentence_embeddings)
mr = collection.insert([em])
ids = mr.primary_keys
dicts ={}

In [ ]:
em

In [ ]:
mr

In [ ]:
ids

In [ ]:
import os 

def record_temp_csv(fname, ids, title, text):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + title[i] + "|" + text[i] + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    try:
        sql = "COPY " + table_name + " FROM STDIN DELIMITER '|' CSV HEADER"
        cursor.copy_expert(sql, open(fname, "r"))
        conn.commit()
        print("Inserted into Postgress Sucessfully!")
    except Exception as e:
        print("Copy Data into Postgress failed: ", e)
        
DATA_WITH_IDS = 'data/test.csv'   

record_temp_csv(DATA_WITH_IDS, ids, title_data, text_data)
copy_data_to_pg(TABLE_NAME, DATA_WITH_IDS, conn, cursor)

In [ ]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

query_vec = []

title = "Loosing the War on Terrorism"

query_embeddings = []
embed = model.encode(title)
embed = embed.reshape(1,-1)
embed = normalize(embed)
query_embeddings = embed.tolist()

collection.load()
results = collection.search(query_embeddings, field_name, param=search_params, limit=9, expr=None)

In [ ]:
similar_titles = []

if results[0][0].distance < 0.5:
    print("There are no similar questions in the database, here are the closest matches:")
else:
    print("There are similar questions in the database, here are the closest matches: ")
    
for result in results[0]:
    sql = "select title from " + TABLE_NAME + " where ids = " + str(result.id) + ";"
    cursor.execute(sql)
    rows=cursor.fetchall()
    if len(rows):
        similar_titles.append((rows[0][0], result.distance))
        print((rows[0][0], result.distance))

In [ ]:
sql = "select text from " + TABLE_NAME + " where title = '" + similar_titles[0][0] + "';"
cursor.execute(sql)
rows=cursor.fetchall()
print("Title:")
print(title)
print("Text:")
print(rows[0][0])